# IMPORTS

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 

# LAB DATA

 ### Tabla general de nombre de variables y de los archivos de los datos

In [ ]:
# URL de ejemplo
url = "https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Laboratory&Cycle=2021-2023"

# Obtener el contenido HTML de la página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

table = soup.find("table", id="GridView1")  # Buscar por ID

# Extraer encabezados (th)
headers = [header.text.strip() for header in table.find_all("th")]

# Extraer filas de datos (td)
rows = []
for row in table.find_all("tr"):
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    if cols:  # Ignorar filas vacías
        rows.append(cols)

# Crear un DataFrame con los encabezados y las filas
df_samplename = pd.DataFrame(rows, columns=headers)

# Mostrar o guardar el DataFrame
df_samplename.to_csv("./data/Scrapping/Datasheet.csv", index=False)
df_samplename

,Data File Name,Doc File,Data File,Date Published
0,alpha-1-Acid Glycoprotein,AGP_L Doc,AGP_L Data [XPT - 61.3 KB],September 2024
1,Cholesterol – High-Density Lipoprotein,HDL_L Doc,HDL_L Data [XPT - 253.4 KB],September 2024
2,Cholesterol - Total,TCHOL_L Doc,TCHOL_L Data [XPT - 253.4 KB],September 2024
3,Complete Blood Count with 5-Part Differential ...,CBC_L Doc,CBC_L Data [XPT - 1.5 MB],September 2024
4,Fasting Questionnaire,FASTQX_L Doc,FASTQX_L Data [XPT - 1.3 MB],September 2024
5,Ferritin,FERTIN_L Doc,FERTIN_L Data [XPT - 81.4 KB],September 2024
6,Folate - RBC,FOLATE_L Doc,FOLATE_L Data [XPT - 274 KB],September 2024
7,Glycohemoglobin,GHB_L Doc,GHB_L Data [XPT - 169.9 KB],September 2024
8,Hepatitis A,HEPA_L Doc,HEPA_L Data [XPT - 203 KB],September 2024
9,Hepatitis B Surface Antibody,HEPB_S_L Doc,HEPB_S_L Data [XPT - 203 KB],September 2024


### Generar una lista con los nombres de variables para posteriormente utilizarla para descargar más datos del apartado de laboratorio

In [ ]:
temp = []
variable_names_cleaned = []
variable_names = df_samplename["Doc File"].to_list()
for var in variable_names:
    clean_var = var.strip()[:-3]
    temp.append(clean_var)
    
for var in temp:
    x = var.replace(" ","")
    variable_names_cleaned.append(x)

variable_names_cleaned

['AGP_L',
 'HDL_L',
 'TCHOL_L',
 'CBC_L',
 'FASTQX_L',
 'FERTIN_L',
 'FOLATE_L',
 'GHB_L',
 'HEPA_L',
 'HEPB_S_L',
 'HSCRP_L',
 'INS_L',
 'PBCD_L',
 'IHGEM_L',
 'GLU_L',
 'FOLFMS_L',
 'TST_L',
 'TFR_L',
 'UCPREG_L',
 'VID_L']

### Sacar tabla con el nombre de las variables, su target y su id

In [ ]:
all_data = []

# URL de ejemplo
for x in variable_names_cleaned:
    try:
        url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/{x}.htm"
        # Obtener el contenido HTML de la página
        response = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
        soup = BeautifulSoup(response.content, "html.parser")

        # Buscar la tabla en el HTML
        table = soup.find_all("dl")
        data = []

        # Buscar todas las listas <dl>
        dl_blocks = soup.find_all("dl")
        data = []

        # Iterar sobre cada bloque <dl>
        for dl in dl_blocks:
            block_data = {}
            # Extraer pares <dt> y <dd>
            dt_tags = dl.find_all("dt")
            dd_tags = dl.find_all("dd")
            
            # Emparejar <dt> (clave) con <dd> (valor)
            for dt, dd in zip(dt_tags, dd_tags):
                key = dt.text.strip().replace(":", "")  # Limpiar el texto de <dt>
                value = dd.text.strip()  # Limpiar el texto de <dd>
                block_data[key] = value  # Crear un diccionario
            
            # Agregar el diccionario a la lista
            data.append(block_data)

        # Añadir los datos obtenidos de esta página a la lista total
        all_data.extend(data)
        
        # Dormir por un tiempo para evitar hacer demasiadas peticiones rápidamente
        time.sleep(3)  # Pausa de 2 segundos entre cada solicitud

    except Exception as e:
        print(f"Error al acceder a {url}: {e}")

# Convertir la lista de diccionarios en un DataFrame
df_descriptions = pd.DataFrame(all_data)

# Limpiar la columna 'Target' para remover espacios adicionales
df_descriptions['Target'] = df_descriptions['Target'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Guardar como archivo CSV
df_descriptions.to_csv("./data/Scrapping/variables_descriptions.csv", index=False)

# Mostrar el DataFrame
df_descriptions

,Variable Name,SAS Label,English Text,Target,English Instructions,Hard Edits
0,SEQN,Respondent sequence number,Respondent sequence number.,Females only 12 YEARS - 49 YEARS,NaN,NaN
1,WTPH2YR,Phlebotomy 2 Year Weight,Phlebotomy 2 Year Weight,Females only 12 YEARS - 49 YEARS,NaN,NaN
2,LBXAGP,alpha-1-acid glycoprotein (g/L),alpha-1-acid glycoprotein (g/L),Females only 12 YEARS - 49 YEARS,NaN,NaN
3,SEQN,Respondent Sequence Number,Respondent Sequence Number,Both males and females 6 YEARS - 150 YEARS,NaN,NaN
4,WTPH2YR,Phlebotomy 2 Year Weight,Phlebotomy 2 Year Weight,Both males and females 6 YEARS - 150 YEARS,NaN,NaN
...,...,...,...,...,...,...
172,LBDVD2LC,25OHD2 comment code,25-hydroxyvitamin D2 comment code,Both males and females 1 YEARS - 150 YEARS,NaN,NaN
173,LBXVD3MS,25OHD3 (nmol/L),25-hydroxyvitamin D3 (nmol/L),Both males and females 1 YEARS - 150 YEARS,NaN,NaN
174,LBDVD3LC,25OHD3 comment code,25-hydroxyvitamin D3 comment code,Both males and females 1 YEARS - 150 YEARS,NaN,NaN
175,LBXVE3MS,epi-25OHD3 (nmol/L),epi-25-hydroxyvitamin D3 (nmol/L),Both males and females 1 YEARS - 150 YEARS,NaN,NaN


### Sacar tablas con la descripción y rango de unidades de las variables

In [ ]:
# Diccionario para almacenar los DataFrames
dataframes = {}

# Recorrer todas las páginas de variable_names_cleaned
for x in variable_names_cleaned:
    try:
        # Construir la URL usando el valor de la variable
        url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/{x}.htm"

        # Obtener el contenido HTML de la página
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.content, "html.parser")

        # Buscar todos los bloques <div class="pagebreak">
        pagebreaks = soup.find_all("div", class_="pagebreak")
        
        # Inicializar un diccionario para almacenar las tablas por nombre
        table_dataframes = {}

        # Iterar sobre los bloques de cada página
        for block in pagebreaks:
            # Extraer el título dentro de <h3 id="ID"> (este es el nombre de la tabla)
            title_tag = block.find("h3", class_="vartitle")
            if title_tag:
                table_name = title_tag.get("id")  # El id será el nombre de la tabla

                # Buscar las tablas dentro de cada bloque
                tables = block.find_all("table", class_="values")
                
                # Inicializar una lista para almacenar los datos de esta tabla
                for idx, table in enumerate(tables):
                    # Extraer encabezados (th) y filas (tr)
                    headers = [th.text.strip() for th in table.find_all("th")]
                    rows = table.find_all("tr")

                    # Lista para almacenar los datos de esta tabla
                    all_data = []

                    for row in rows:
                        cols = row.find_all("td")
                        cols = [ele.text.strip() for ele in cols]
                        if cols:  # Evitar agregar filas vacías
                            all_data.append(cols)

                    # Solo agregar la tabla si tiene datos
                    if all_data:
                        df = pd.DataFrame(all_data, columns=headers)
                        
                        # Guardar el DataFrame en el diccionario con el nombre extraído
                        table_dataframes[table_name] = df

        # Guardar las tablas del DataFrame en el diccionario de dataframes
        dataframes[x] = table_dataframes

        # Dormir por un tiempo para evitar hacer demasiadas peticiones rápidamente
        time.sleep(3)  # Pausa de 3 segundos entre cada solicitud

    except Exception as e:
        print(f"Error al acceder a {url}: {e}")

# Mostrar los DataFrames guardados
for variable_name, tables in dataframes.items():
    print(f"Datos para {variable_name}:")
    for table_name, df in tables.items():
        print(f"\nTabla: {table_name}")
        print(df.head())  # Mostrar las primeras filas de cada tabla

# Si lo deseas, guardar cada DataFrame de cada tabla en un archivo CSV
for variable_name, tables in dataframes.items():
    for table_name, df in tables.items():
        df.to_csv(f"./data/Scrapping/{variable_name}_{table_name}_data.csv", index=False)

Datos para AGP_L:

Tabla: WTPH2YR
                  Code or Value         Value Description Count Cumulative  \
0  6213.3002375 to 253478.77765           Range of Values  2051       2051   
1                             0  No blood sample provided   513       2564   
2                             .                   Missing     0       2564   

  Skip to Item  
0               
1               
2               

Tabla: LBXAGP
   Code or Value Value Description Count Cumulative Skip to Item
0  0.261 to 2.76   Range of Values  1950       1950             
1              .           Missing   614       2564             
Datos para HDL_L:

Tabla: WTPH2YR
                  Code or Value         Value Description Count Cumulative  \
0  4391.8220579 to 241728.85724           Range of Values  7316       7316   
1                             0  No blood sample provided   752       8068   
2                             .                   Missing     0       8068   

  Skip to Item  
0          

# DEMOGRAPHIC DATA

### Obtener la descripcion de variables de Demografia

In [ ]:
url = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.htm"
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.content, "html.parser")

# Buscar todos los bloques <div class="pagebreak">
pagebreaks = soup.find_all("div", class_="pagebreak")

table_dataframes = {}

for block in pagebreaks:
    try:
        # Extraer el título dentro de <h3 id="ID"> (este es el nombre de la tabla)
        title_tag = block.find("h3", class_="vartitle")
        if title_tag:
            table_name = title_tag.get("id")  # El id será el nombre de la tabla

            # Buscar las tablas dentro de cada bloque
            tables = block.find_all("table", class_="values")
            
            # Procesar cada tabla
            for idx, table in enumerate(tables):
                # Extraer encabezados (th) y filas (tr)
                headers = [th.text.strip() for th in table.find_all("th")]
                rows = table.find_all("tr")

                # Lista para almacenar los datos de esta tabla
                all_data = []

                for row in rows:
                    cols = row.find_all("td")  # Sin la clase "values"
                    cols = [ele.text.strip() for ele in cols]
                    if cols:  # Evitar agregar filas vacías
                        all_data.append(cols)

                # Solo agregar la tabla si tiene datos
                if all_data:
                    df = pd.DataFrame(all_data, columns=headers)
                    
                    # Guardar la tabla en el diccionario con un nombre único
                    table_dataframes[table_name + f"_{idx}"] = df

    except Exception as e:
        print(f"Error procesando el bloque: {e}")

# Guardar los DataFrames en CSV
for table_name, df in table_dataframes.items():
    df.to_csv(f"./Demographics/{table_name}_data.csv", index=False)

# QUESTIONNAIRE DATA

### Obtener la descripcion de las variables de los questionarios

In [ ]:
# URL de ejemplo
url = "https://wwwn.cdc.gov/Nchs/Nhanes/Search/DataPage.aspx?Component=Questionnaire&Cycle=2021-2023"

# Obtener el contenido HTML de la página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

table = soup.find("table", id="GridView1")  # Buscar por ID

# Extraer encabezados (th)
headers = [header.text.strip() for header in table.find_all("th")]

# Extraer filas de datos (td)
rows = []
for row in table.find_all("tr"):
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    if cols:  # Ignorar filas vacías
        rows.append(cols)

# Crear un DataFrame con los encabezados y las filas
df_samplename = pd.DataFrame(rows, columns=headers)

# Mostrar o guardar el DataFrame
df_samplenameq.to_csv("./data/Questions/Datasheet.csv", index=False)
df_samplenameq

,Data File Name,Doc File,Data File,Date Published
0,Audiometry,AUQ_L Doc,AUQ_L Data [XPT - 1.3 MB],September 2024
1,Acculturation,ACQ_L Doc,ACQ_L Data [XPT - 445.6 KB],September 2024
2,Alcohol Use,ALQ_L Doc,ALQ_L Data [XPT - 447.6 KB],September 2024
3,Balance,BAQ_L Doc,BAQ_L Data [XPT - 714.1 KB],October 2024
4,Blood Pressure & Cholesterol,BPQ_L Doc,BPQ_L Data [XPT - 400.1 KB],September 2024
5,Current Health Status,HSQ_L Doc,HSQ_L Data [XPT - 104.4 KB],September 2024
6,Dermatology,DEQ_L Doc,DEQ_L Data [XPT - 135.8 KB],September 2024
7,Diabetes,DIQ_L Doc,DIQ_L Data [XPT - 827.7 KB],September 2024
8,Diet Behavior & Nutrition,DBQ_L Doc,DBQ_L Data [XPT - 2.5 MB],September 2024
9,Early Childhood,ECQ_L Doc,ECQ_L Data [XPT - 189.5 KB],September 2024


### Generar una lista con los nombres de variables para posteriormente utilizarla para descargar más datos del apartado de questionarios

In [ ]:
temp = []
variable_names_cleanedq = []
variable_names = df_samplenameq["Doc File"].to_list()
for var in variable_names:
    clean_var = var.strip()[:-3]
    temp.append(clean_var)
    
for var in temp:
    x = var.replace(" ","")
    variable_names_cleanedq.append(x)

variable_names_cleanedq

['AUQ_L',
 'ACQ_L',
 'ALQ_L',
 'BAQ_L',
 'BPQ_L',
 'HSQ_L',
 'DEQ_L',
 'DIQ_L',
 'DBQ_L',
 'ECQ_L',
 'FNQ_L',
 'HIQ_L',
 'HEQ_L',
 'HUQ_L',
 'HOQ_L',
 'IMQ_L',
 'INQ_L',
 'KIQ_U_L',
 'MCQ_L',
 'DPQ_L',
 'OCQ_L',
 'OHQ_L',
 'PUQMEC_L',
 'PAQ_L',
 'PAQY_L',
 'RXQ_RX_L',
 'RXQASA_L',
 'RHQ_L',
 'SLQ_L',
 'SMQ_L',
 'SMQFAM_L',
 'SMQRTU_L',
 'WHQ_L']

### Descaga de los datos

### Descargo los datos de laboratorio de los pacientes en formato .xpt, los transformo a formato csv y borro los datos .xpt

In [ ]:

# URL base para los archivos XPT
url_base = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/"

# Ruta de guardado local donde se guardarán los archivos descargados
save_directory = "data/laboratory/"

# Asegúrate de que el directorio exista
os.makedirs(save_directory, exist_ok=True)

# Descargar cada archivo XPT y convertirlo a CSV
for endpoint in variable_names_cleaned:
    try:
        # Construir la URL completa para el archivo XPT
        url = url_base + f"{endpoint}.xpt"
        
        # Definir la ruta de guardado del archivo XPT
        xpt_file_path = os.path.join(save_directory, f"{endpoint}.xpt")
        
        # Realizar la solicitud para obtener el archivo
        response = requests.get(url)
        
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Guardar el archivo XPT en el disco
            with open(xpt_file_path, 'wb') as file:
                file.write(response.content)
            print(f"Archivo {endpoint}.xpt guardado en: {xpt_file_path}")

            # Leer el archivo .xpt usando pandas
            data = pd.read_sas(xpt_file_path, format='xport')

            # Mostrar las primeras filas del DataFrame
            print(f"Primeras filas de {endpoint}:\n", data.head())

            # Guardar el DataFrame como CSV
            csv_file_path = os.path.join(save_directory, f"{endpoint}.csv")
            data.to_csv(csv_file_path, index=False)
            print(f"Archivo CSV guardado en: {csv_file_path}")

            # Eliminar el archivo XPT original
            os.remove(xpt_file_path)
            print(f"Archivo {endpoint}.xpt eliminado.")

        else:
            print(f"Error al descargar {url}: {response.status_code}")
    
    except Exception as e:
        print(f"Hubo un error al intentar descargar o procesar {url}: {e}")


Archivo AGP_L.xpt guardado en: data/laboratory/AGP_L.xpt
Primeras filas de AGP_L:
        SEQN       WTPH2YR  LBXAGP
0  130380.0  8.532884e+04   1.010
1  130381.0  5.397605e-79     NaN
2  130382.0  5.963893e+04   0.921
3  130388.0  3.286429e+04     NaN
4  130390.0  6.142701e+04   1.040
Archivo CSV guardado en: data/laboratory/AGP_L.csv
Archivo AGP_L.xpt eliminado.
Archivo HDL_L.xpt guardado en: data/laboratory/HDL_L.xpt
Primeras filas de HDL_L:
        SEQN       WTPH2YR  LBDHDD  LBDHDDSI
0  130378.0  56042.129410    45.0      1.16
1  130379.0  37435.705647    60.0      1.55
2  130380.0  85328.844519    49.0      1.27
3  130386.0  44526.214135    46.0      1.19
4  130387.0  22746.296353    42.0      1.09
Archivo CSV guardado en: data/laboratory/HDL_L.csv
Archivo HDL_L.xpt eliminado.
Archivo TCHOL_L.xpt guardado en: data/laboratory/TCHOL_L.xpt
Primeras filas de TCHOL_L:
        SEQN       WTPH2YR  LBXTC  LBDTCSI
0  130378.0  56042.129410  264.0     6.83
1  130379.0  37435.705647  214.0 

### Descargo los datos de questionarios de los pacientes en formato .xpt, los transformo a formato csv y borro los datos .xpt

In [ ]:

# URL base para los archivos XPT
url_base = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/"

# Ruta de guardado local donde se guardarán los archivos descargados
save_directory = "data/Questions/"

# Asegúrate de que el directorio exista
os.makedirs(save_directory, exist_ok=True)

# Descargar cada archivo XPT y convertirlo a CSV
for endpoint in variable_names_cleanedq:
    try:
        # Construir la URL completa para el archivo XPT
        url = url_base + f"{endpoint}.xpt"
        
        # Definir la ruta de guardado del archivo XPT
        xpt_file_path = os.path.join(save_directory, f"{endpoint}.xpt")
        
        # Realizar la solicitud para obtener el archivo
        response = requests.get(url)
        
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Guardar el archivo XPT en el disco
            with open(xpt_file_path, 'wb') as file:
                file.write(response.content)
            print(f"Archivo {endpoint}.xpt guardado en: {xpt_file_path}")

            # Leer el archivo .xpt usando pandas
            data = pd.read_sas(xpt_file_path, format='xport')

            # Mostrar las primeras filas del DataFrame
            print(f"Primeras filas de {endpoint}:\n", data.head())

            # Guardar el DataFrame como CSV
            csv_file_path = os.path.join(save_directory, f"{endpoint}.csv")
            data.to_csv(csv_file_path, index=False)
            print(f"Archivo CSV guardado en: {csv_file_path}")

            # Eliminar el archivo XPT original
            os.remove(xpt_file_path)
            print(f"Archivo {endpoint}.xpt eliminado.")

        else:
            print(f"Error al descargar {url}: {response.status_code}")
    
    except Exception as e:
        print(f"Hubo un error al intentar descargar o procesar {url}: {e}")


Archivo AUQ_L.xpt guardado en: data/Questions/AUQ_L.xpt
Primeras filas de AUQ_L:
        SEQN  AUQ054  AUQ410A  AUQ410B  AUQ410C  AUQ410D  AUQ410E  AUQ410F  \
0  130378.0     1.0      NaN      NaN      NaN      NaN      NaN      NaN   
1  130379.0     4.0      NaN      NaN      NaN      NaN      NaN      NaN   
2  130380.0     4.0      NaN      NaN      NaN      NaN      NaN      NaN   
3  130381.0     1.0      NaN      NaN      NaN      NaN      NaN      NaN   
4  130382.0     1.0      NaN      NaN      NaN      NaN      NaN      NaN   

   AUQ410G  AUQ410H  AUQ410I  AUQ410J  AUQ144  AUQ101  
0      NaN      NaN      NaN      NaN     1.0     5.0  
1      7.0      NaN      9.0      NaN     1.0     3.0  
2      7.0      NaN      NaN      NaN     2.0     1.0  
3      NaN      NaN      NaN      NaN     NaN     NaN  
4      NaN      NaN      NaN      NaN     NaN     NaN  
Archivo CSV guardado en: data/Questions/AUQ_L.csv
Archivo AUQ_L.xpt eliminado.
Archivo ACQ_L.xpt guardado en: data/Ques

### Descargo todos los datos de información de peso y altura de todos los años disponibles

In [ ]:
import os
import requests
import pandas as pd

# Parámetros
all_dates = ["2017", "2015", "2013", "2011", "2009", "2007", "2005", "2003", "2001", "1999"]
letters = ["J", "I", "H", "G", "F", "E", "D", "C", "B"]

# Carpeta para guardar los archivos
output_folder = "./data/Questions/Historic/"
os.makedirs(output_folder, exist_ok=True)

# Headers para la solicitud
headers = {"User-Agent": "Mozilla/5.0"}

# Bucle para recorrer las fechas y las letras
for idx, year in enumerate(all_dates):
    if year == "1999":
        # Caso especial para 1999 (sin letra)
        url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{year}/DataFiles/WHQ_.xpt"
    else:
        # Caso general para otros años
        letter = letters[idx]
        url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{year}/DataFiles/WHQ_{letter}.xpt"

    print(f"Procesando: {url}")

    # Hacer la solicitud a la URL
    response = requests.get(url, headers=headers)

    # Comprobar si la solicitud fue exitosa
    if response.status_code == 200:
        # Guardar el archivo en la carpeta de descargas
        filename = f"WHQ_{year}.xpt" if year == "1999" else f"WHQ_{year}_{letter}.xpt"
        filepath = os.path.join(output_folder, filename)
        with open(filepath, "wb") as f:
            f.write(response.content)
        print(f"Descargado: {filename}")

        # Convertir el archivo XPT a CSV
        try:
            df = pd.read_sas(filepath, format="xport")
            csv_filename = filename.replace(".xpt", ".csv")
            csv_filepath = os.path.join(output_folder, csv_filename)
            df.to_csv(csv_filepath, index=False)
            print(f"Convertido a CSV: {csv_filename}")

            # Eliminar el archivo XPT original
            os.remove(filepath)
            print(f"Archivo XPT eliminado: {filename}")
        except Exception as e:
            print(f"Error al convertir {filename} a CSV: {e}")
    else:
        print(f"Error {response.status_code} al intentar descargar {url}")

# Manejo del archivo especial P_WHQMEC.xpt
special_url = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/P_WHQ.xpt"
print(f"Procesando archivo especial: {special_url}")

response = requests.get(special_url, headers=headers)

if response.status_code == 200:
    special_filename = "P_WHQMEC.xpt"
    special_filepath = os.path.join(output_folder, special_filename)
    with open(special_filepath, "wb") as f:
        f.write(response.content)
    print(f"Descargado archivo especial: {special_filename}")

    # Convertir a CSV
    try:
        df = pd.read_sas(special_filepath, format="xport")
        special_csv_filename = special_filename.replace(".xpt", ".csv")
        special_csv_filepath = os.path.join(output_folder, special_csv_filename)
        df.to_csv(special_csv_filepath, index=False)
        print(f"Convertido archivo especial a CSV: {special_csv_filename}")

        # Eliminar el archivo XPT original
        os.remove(special_filepath)
        print(f"Archivo XPT especial eliminado: {special_filename}")
    except Exception as e:
        print(f"Error al convertir archivo especial {special_filename} a CSV: {e}")
else:
    print(f"Error {response.status_code} al intentar descargar archivo especial {special_url}")


Procesando: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/WHQ_J.xpt
Descargado: WHQ_2017_J.xpt
Convertido a CSV: WHQ_2017_J.csv
Archivo XPT eliminado: WHQ_2017_J.xpt
Procesando: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2015/DataFiles/WHQ_I.xpt
Descargado: WHQ_2015_I.xpt
Convertido a CSV: WHQ_2015_I.csv
Archivo XPT eliminado: WHQ_2015_I.xpt
Procesando: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/WHQ_H.xpt
Descargado: WHQ_2013_H.xpt
Convertido a CSV: WHQ_2013_H.csv
Archivo XPT eliminado: WHQ_2013_H.xpt
Procesando: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2011/DataFiles/WHQ_G.xpt
Descargado: WHQ_2011_G.xpt
Convertido a CSV: WHQ_2011_G.csv
Archivo XPT eliminado: WHQ_2011_G.xpt
Procesando: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2009/DataFiles/WHQ_F.xpt
Descargado: WHQ_2009_F.xpt
Convertido a CSV: WHQ_2009_F.csv
Archivo XPT eliminado: WHQ_2009_F.xpt
Procesando: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2007/DataFiles/WHQ_E.xpt
Descargado: WHQ_20